# Part 2: Explore data

In this tutorial, you'll use `seaborn`, which is a Python data visualization library that provides a high-level interface for building visuals on dataframes and arrays. For more information about `seaborn`, see [seaborn: statistical data visualization](https://seaborn.pydata.org/).  

You'll also use [Data Wrangler]((https://aka.ms/fabric/datawrangler)), a notebook-based tool that provide you with an immersive experience to conduct exploratory data analysis and cleansing.

In this tutorial you learn to perform the following actions:

1. Read the data stored from a delta table in the lakehouse.
1. Convert a Spark DataFrame to Pandas DataFrame, which python visualization libraries support.
1. Use Data Wrangler to perform initial data cleaning and transformation.
1. Perform exploratory data analysis using `seaborn`.


## Prerequisites

- Complete [Part 1: Ingest data](https://learn.microsoft.com/fabric/data-science/tutorial-data-science-ingest-data).
- Attach the same lakehouse you used in Part 1 to this notebook.

## Read raw data from the lakehouse

Read raw data from the **Files** section of the lakehouse. You uploaded this data in the previous notebook. Make sure you have attached the same lakehouse you used in Part 1 to this notebook before you run this code.

In [ ]:
df = (
    spark.read.option("header", True)
    .option("inferSchema", True)
    .csv("Files/churn/raw/churn.csv")
    .cache()
)

## Create a pandas DataFrame from the dataset

Convert the spark DataFrame to pandas DataFrame for easier processing and visualization.

In [ ]:
df = df.toPandas()

### Display raw data

Explore the raw data with `display`, do some basic statistics and show chart views. You first need to import required libraries for data visualization such as `seaborn`, which is a Python data visualization library to provide a high-level interface for building visuals on DataFrames and arrays. Learn more about [`seaborn`](https://seaborn.pydata.org/). 

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid", palette="tab10", rc = {'figure.figsize':(9,6)})
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import rc, rcParams
import numpy as np
import pandas as pd
import itertools

In [ ]:
display(df, summary=True)

## Use Data Wrangler to perform initial data cleansing

Data Wrangler is a notebook-based tool that provides an immersive experience to conduct exploratory data analysis and cleansing. 

To use Data Wrangler for performing initial data cleansing, follows the steps in [the online documentation](https://learn.microsoft.com/fabric/data-science/tutorial-data-science-explore-notebook#data-wrangler). 

Once you complete these steps, you'll see a new notebook code cell here, which was generated by Data Wrangler. Run that code cell to perform the operations you explored in Data Wrangler.

If you didn't use Data Wrangler, you can instead use this next code cell. 

This code is similar to the code produced by Data Wrangler, but adds in the argument `inplace=True` to each of the generated steps. By setting `inplace=True`, pandas will overwrite the original DataFrame instead of producing a new DataFrame as an output.

In [ ]:
# Modified version of code generated by Data Wrangler 
# Modification is to add in-place=True to each step

# Define a new function that include all above Data Wrangler operations
def clean_data(df):
    # Drop rows with missing data across all columns
    df.dropna(inplace=True)
    # Drop duplicate rows in columns: 'RowNumber', 'CustomerId'
    df.drop_duplicates(subset=['RowNumber', 'CustomerId'], inplace=True)
    # Drop columns: 'RowNumber', 'CustomerId', 'Surname'
    df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)
    return df

df_clean = clean_data(df.copy())
df_clean.head()

## Explore the data

Display some summaries and visualizations of the cleaned data.

### Determine categorical, numerical, and target attributes

Use this code to determine categorical, numerical, and target attributes.

In [ ]:
# Determine the dependent (target) attribute
dependent_variable_name = "Exited"
print(dependent_variable_name)
# Determine the categorical attributes
categorical_variables = [col for col in df_clean.columns if col in "O"
                        or df_clean[col].nunique() <=5
                        and col not in "Exited"]
print(categorical_variables)
# Determine the numerical attributes
numeric_variables = [col for col in df_clean.columns if df_clean[col].dtype != "object"
                        and df_clean[col].nunique() >5]
print(numeric_variables)

### The five-number summary 

Show the five-number summary (the minimum score, first quartile, median, third quartile, the maximum score) for the numerical attributes, using box plots.

In [ ]:
df_num_cols = df_clean[numeric_variables]
sns.set(font_scale = 0.7) 
fig, axes = plt.subplots(nrows = 2, ncols = 3, gridspec_kw =  dict(hspace=0.3), figsize = (17,8))
fig.tight_layout()
for ax,col in zip(axes.flatten(), df_num_cols.columns):
    sns.boxplot(x = df_num_cols[col], color='green', ax = ax)
fig.delaxes(axes[1,2])


## Distribution of exited and nonexited customers 

Show the distribution of exited versus nonexited customers across the categorical attributes. 

In [ ]:
attr_list = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember', 'NumOfProducts', 'Tenure']
fig, axarr = plt.subplots(2, 3, figsize=(15, 4))
for ind, item in enumerate (attr_list):
    sns.countplot(x = item, hue = 'Exited', data = df_clean, ax = axarr[ind%2][ind//2])
fig.subplots_adjust(hspace=0.7)

### Distribution of numerical attributes

Show the frequency distribution of numerical attributes using histogram. 

In [ ]:
columns = df_num_cols.columns[: len(df_num_cols.columns)]
fig = plt.figure()
fig.set_size_inches(18, 8)
length = len(columns)
for i,j in itertools.zip_longest(columns, range(length)):
    plt.subplot((length // 2), 3, j+1)
    plt.subplots_adjust(wspace = 0.2, hspace = 0.5)
    df_num_cols[i].hist(bins = 20, edgecolor = 'black')
    plt.title(i)
plt.show()

## Perform feature engineering

Perform feature engineering to generate new attributes based on current attributes:

In [ ]:
df_clean["NewTenure"] = df_clean["Tenure"]/df_clean["Age"]
df_clean["NewCreditsScore"] = pd.qcut(df_clean['CreditScore'], 6, labels = [1, 2, 3, 4, 5, 6])
df_clean["NewAgeScore"] = pd.qcut(df_clean['Age'], 8, labels = [1, 2, 3, 4, 5, 6, 7, 8])
df_clean["NewBalanceScore"] = pd.qcut(df_clean['Balance'].rank(method="first"), 5, labels = [1, 2, 3, 4, 5])
df_clean["NewEstSalaryScore"] = pd.qcut(df_clean['EstimatedSalary'], 10, labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

Data Wrangler can also be used to perform one-hot encoding.  To do so, re-open Data Wrangler.  This time, select the `df_clean` data. Then select **One-hot encode** under **Formulas**.  

Or use this code:

In [ ]:
# This is the same code that Data Wrangler will generate
 
import pandas as pd
 
def clean_data(df_clean):
    # One-hot encode columns: 'Geography', 'Gender'
    df_clean = pd.get_dummies(df_clean, columns=['Geography', 'Gender'])
    return df_clean
 
df_clean_1 = clean_data(df_clean.copy())
df_clean_1.head()

## Summary of observations from the exploratory data analysis

- Most of the customers are from France comparing to Spain and Germany, while Spain has the lowest churn rate comparing to France and Germany.
- Most of the customers have credit cards.
- There are customers whose age and credit score are above 60 and below 400, respectively, but they can't be considered as outliers.
- Very few customers have more than two of the bank's products.
- Customers who aren't active have a higher churn rate.
- Gender and tenure years don't seem to have an impact on customer's decision to close the bank account.

## Create a delta table for the cleaned data

You'll use this data in the next notebook of this series.

In [ ]:
table_name = "df_clean"
# Create Spark DataFrame from pandas
sparkDF=spark.createDataFrame(df_clean_1) 
sparkDF.write.mode("overwrite").format("delta").save(f"Tables/{table_name}")
print(f"Spark dataframe saved to delta table: {table_name}")

## Next step

Proceed to [Part 3: Train and register machine learning models](https://learn.microsoft.com/fabric/data-science/tutorial-data-science-train-models) to use the `df_clean` data to train and register machine learning models.